# General
This notebook goal is to create a general policy for every kind of speed, terrain and locomotion.

What I will need to change:

- **Terrain** quality and variety (shape and properties)
- Random **inputs** for the robot mimiking a controller
- depth of the **network**
- (optional) different way of **training** it, like evolving algorythms



# Terrain

To improve the terrain I need to:
- understand how terrain is managed in pybullet
- understand ways of randomizing it
- Test it
- Find a nice way to implement randomization during the simulation

## How terrain is managed in pybullet

There are three main ways to handle terrain in pybullet:

1. **Infinite Plane** (`GEOM_PLANE`): This is the most basic ground. It's a perfectly flat, infinite plane. It's great for simple tests but not for realistic scenarios.
2. **Heightfield** (`GEOM_HEIGHTFIELD`): This is the most powerful and common method for creating complex, realistic terrains. A heightfield is essentially a 2D grid where each point's value represents the height of the terrain at that (x, y) coordinate. Imagine a grayscale image where white is the highest peak and black is the lowest valley.
3. **Triangle Mesh** (`GEOM_MESH`): If you have a terrain model created in 3D software (like Blender) and saved as a file (e.g., .obj, .stl), you can load it directly. This is great for highly specific, non-grid-based terrains but can be less performant than a heightfield.

Questions: are this the only ways to handle terrain in pybullet?

In any case we will use heightfield because is the most efficient and easily randomizable method of the three.

### Heightfield


#### Step 1: Setting Up the Environment
First, we need to import the necessary libraries and connect to the PyBullet physics engine. We will need:

- *pybullet* for the simulation itself.
- *numpy* to create and manipulate the numerical data for our heightfield.
- *time* to run the simulation at a human-viewable speed.
We will connect to PyBullet using pybullet.GUI, which will open a window to visualize the simulation.

In [ ]:
import pybullet
import numpy as np
import time

# Connect to the physics server. The GUI option opens a visualization window.
# If you want to run it without a window (e.g., for training), you can use pybullet.DIRECT.
physics_client_id = pybullet.connect(pybullet.GUI)

# Set a gravitational force.
pybullet.setGravity(0, 0, -9.81)

print(f"Connected to PyBullet with client ID: {physics_client_id}")

#### Step 2: Generating the Heightfield Data
We will use NumPy to create a 2D grid of height values. To start, let's make a simple ramp. This will help you visualize how the data corresponds to the shape in the simulation.

1. **Define Dimensions**: We'll specify the number of rows and columns for our terrain grid.
2. **Create a 2D Array**: We'll create a 2D NumPy array and fill it with values that increase along one axis to form a ramp.
3. **Flatten the Array**: The PyBullet function we will use later expects a 1D list of all the height values, not a 2D grid. So, we'll convert our 2D array into a 1D list.


In [ ]:
num_rows = 200
num_columns = 200
height_max = 2
height_min = 0

heightmap = np.zeros((num_rows, num_columns))

# Staircase (used for testing)
height_current = height_min
for i in range(0, num_columns, 2):
    heightmap[:, i-1] = height_current
    heightmap[:, i] = height_current
    height_current += (height_max-height_min)/(num_columns-1)

heightmap

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(heightmap,
               cmap="terrain",
               extent=[0, num_columns, 0, num_rows])
ax.set_title("2D Visualization of the Ramp Heightmap")
ax.set_xlabel("Columns")
ax.grid(True)
ax.set_ylabel("Rows")
# 2. Create a colorbar for the figure, linked to our image 'im'.
# We can also give the colorbar its own label.
fig.colorbar(im, ax=ax, label="Height")

plt.show()

#### Step 3: Creating the Terrain in PyBullet
Creating a physical object from a heightfield is a two-step process:

1. **Create a Collision Shape**: First, you define the geometry of the object. You pass your flattened height data and dimensions to `pybullet.createCollisionShape`. This function returns an ID for the shape.
2. **Create a Body**: Then, you create a physical body using that shape ID with `pybullet.createMultiBody`. This is where you can define properties like mass. For a terrain, we'll set the mass to zero to make it a static, immovable object.

A new and important parameter here is `meshScale`. It's a list of three numbers `[scale_x, scale_y, scale_z]` that determines the real-world size of your terrain:

- `scale_x`: The distance in meters between adjacent columns in your grid.
- `scale_y`: The distance in meters between adjacent rows in your grid.
- `scale_z`: A multiplier for all your height values.

Let's add the code. In a new cell, we will first flatten the `heightmap_2d` array (in case you changed it) and then use it to create the terrain.

In [ ]:
heightmap_flat = heightmap.flatten().tolist()
heightmap_scale = [0.05, 0.05, 1]

terrain_shape_id = pybullet.createCollisionShape(
    shapeType=pybullet.GEOM_HEIGHTFIELD,
    meshScale=heightmap_scale,
    heightfieldData=heightmap_flat, # MISSING from stubs documentation
    numHeightfieldRows=num_rows,
    numHeightfieldColumns=num_columns
)

terrain_body_id = pybullet.createMultiBody(
    baseMass=0,
    baseCollisionShapeIndex=terrain_shape_id
)

pybullet.changeVisualShape(terrain_body_id, -1, rgbaColor=[0.6, 0.6, 0.6, 1])

print(f"Created terrain with body ID: {terrain_body_id}")

pybullet.resetDebugVisualizerCamera(
    cameraDistance=5,
    cameraYaw=35,
    cameraPitch=-30,
    cameraTargetPosition=[0, 0, 0]
)

pybullet.stepSimulation()

# Create Randomized Terrain

- Terrain Layers
    - Optional sub-layers
- Obstacle Layers
  - Optional sub-layers



In [ ]:
import numpy as np
import numpy as np
from pyfastnoiselite.pyfastnoiselite import FastNoiseLite, NoiseType, FractalType
import matplotlib.pyplot as plt

In [ ]:
def generate_height_map(x,
                        y,
                        z_max=1,
                        scale=100,
                        octaves=5,
                        gain=0.5,
                        lacunarity=2.0,
                        seed=np.random.randint(0, 10000)):
    """
    Genera una heightmap 2D utilizzando pyfastnoiselite.

    Args:
        x (int): Larghezza della mappa (numero di colonne).
        y (int): Altezza della mappa (numero di righe).
        z_max (float, optional): L'altezza massima desiderata. 
                                 La mappa finale sarà normalizzata tra [0, z_max].
        scale (float, optional): La "scala" del rumore. Valori alti = "zoom avanti" 
                                 (feature più grandi, frequenza bassa).
        octaves (int, optional): Numero di strati di rumore sovrapposti 
                                 per aggiungere dettagli.
        gain (float, optional): (Ex 'persistence'). Multiplicatore dell'ampiezza 
                                per ogni ottava successiva. < 1 = dettagli più fievoli.
        lacunarity (float, optional): Multiplicatore della frequenza per ogni 
                                      ottava successiva. > 1 = dettagli più fini.
        seed (int, optional): Seme per la generazione casuale del rumore.
    """
    
    # --- 1. Imposta l'oggetto Noise ---
    # Il costruttore è lo stesso
    noise = FastNoiseLite(seed=seed)
    
    # Usa proprietà invece dei metodi Set*/set_*
    noise.noise_type = NoiseType.NoiseType_OpenSimplex2
    
    # 'scale' è l'inverso della frequenza.
    noise.frequency = 1.0 / scale

    # --- 2. Impostazioni Frattali (per le ottave) ---
    noise.fractal_type = FractalType.FractalType_FBm
    noise.fractal_octaves = octaves
    noise.fractal_lacunarity = lacunarity
    noise.fractal_gain = gain
    
    # --- 3. Generazione ---
    
    # Dobbiamo creare la griglia di coordinate manualmente.
    # 1. Crea i vettori di coordinate per ogni asse
    x_coords = np.arange(x)
    y_coords = np.arange(y)
    
    # 2. Crea una griglia 2D di coordinate (shape: [y, x])
    xx, yy = np.meshgrid(x_coords, y_coords)

    # 3. Genera i valori di rumore in modo vettorializzato con gen_from_coords (forma richiesta: [2, N], dtype float32)
    coords = np.vstack([xx.ravel().astype(np.float32), yy.ravel().astype(np.float32)])
    flat_noise = noise.gen_from_coords(coords)
    
    # 4. Rimodella l'array 1D di output nella nostra forma [y, x]
    heightmap = flat_noise.reshape((y, x))
    
    # --- 4. Normalizzazione ---
    # Il rumore è in [-1, 1]. Lo portiamo a [0, z_max]
    heightmap = (heightmap + 1) / 2 * z_max

    return heightmap

# --- Esempio di utilizzo ---
# map_width = 512
# map_height = 512
# 
# my_map = generate_height_map(map_width, map_height, scale=150.0, octaves=6)
# 
# # Per visualizzarlo velocemente (se hai matplotlib)
# try:
#     import matplotlib.pyplot as plt
#     plt.imshow(my_map, cmap='gray')
#     plt.show()
# except ImportError:
#     print("Matplotlib non trovato. Impossibile visualizzare la mappa.")
#     print(f"Forma della mappa generata: {my_map.shape}")



def view_height_map(heightmap):
    """
    Visualizza la heightmap 2D utilizzando matplotlib con una colormap "terrain".

    Args:
        heightmap (np.ndarray): L'array NumPy 2D contenente i dati di altezza.

    Returns:
        tuple: Una tupla contenente gli oggetti matplotlib generati:
            (fig, ax, im)
            - fig (matplotlib.figure.Figure): La figura principale del plot.
            - ax (matplotlib.axes.Axes): Gli assi del plot.
            - im (matplotlib.image.AxesImage): L'oggetto immagine renderizzato.
    """
    num_rows, num_columns = heightmap.shape
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(heightmap,
                cmap="terrain",
                extent=[0, num_columns, 0, num_rows])
    ax.set_title("2D Visualization of the Heightmap")
    ax.set_xlabel("Columns (y)")
    ax.grid(True)
    ax.set_ylabel("Rows (x)")
    # 2. Create a colorbar for the figure, linked to our image 'im'.
    # We can also give the colorbar its own label.
    fig.colorbar(im, ax=ax, label="Height")
    plt.show()
    return(fig, ax, im)


def generate_terrain(heightmap, scale=[1, 1, 1], origin=[0, 0, 0]):
    """
    Crea un corpo terreno 3D statico nella simulazione PyBullet da una heightmap.

    Utilizza pybullet.createCollisionShape con GEOM_HEIGHTFIELD per
    generare un oggetto con massa 0 (statico) nella simulazione.

    Args:
        heightmap (np.ndarray): L'array NumPy 2D da cui generare il terreno.
        scale (list, optional): Una lista [x, y, z] per scalare la mesh
                                del terreno. Il valore z scala l'altezza.
                                Defaults to [1, 1, 1].
        origin (list, optional): La posizione [x, y, z] di base (centro)
                                 del terreno nel mondo PyBullet.
                                 Defaults to [0, 0, 0].

    Returns:
        int: L'ID univoco (terrain_body_id) del corpo terreno creato in PyBullet.
    """
    heightmap_flat = heightmap.flatten().tolist()
    num_rows, num_columns = heightmap.shape
    terrain_shape_id = pybullet.createCollisionShape(
    shapeType=pybullet.GEOM_HEIGHTFIELD,
    meshScale=scale,
    heightfieldData=heightmap_flat, # MISSING from stubs documentation
    numHeightfieldRows=num_rows,
    numHeightfieldColumns=num_columns
    )

    terrain_body_id = pybullet.createMultiBody(
    baseMass=0,
    baseCollisionShapeIndex=terrain_shape_id,
    basePosition=origin
    )

    pybullet.changeVisualShape(terrain_body_id, -1, rgbaColor=[0.6, 0.6, 0.6, 1])

    print(f"Created terrain with body ID: {terrain_body_id}")
    return terrain_body_id

In [ ]:
heightmap = generate_height_map(200, 200,
                                scale=100,
                                z_max=10,
                                gain=0.5,
                                lacunarity=3,
                                octaves=5)
view_height_map(heightmap)
terrain_body_id=generate_terrain(heightmap, scale=[0.2, 0.2, 0.2])
# View the changes
pybullet.resetDebugVisualizerCamera(
    cameraDistance=5,
    cameraYaw=35,
    cameraPitch=-30,
    cameraTargetPosition=[0, 0, 0]
)

# pybullet.stepSimulation()

In [ ]:
pybullet.removeBody(terrain_body_id)

# Test Finale

In [ ]:
import TerrainTools as tt
import pybullet


physics_client_id = pybullet.connect(pybullet.GUI)
# Set a gravitational force.
pybullet.setGravity(0, 0, -9.81)
print(f"Connected to PyBullet with client ID: {physics_client_id}")

# staircaise_heightmap = tt.Heightmap.from_stairs(200, 200)
random_heightmap = tt.Heightmap.from_noise(z_max=40)
h = tt.Heightmap.from_noise(200, 200, 2, 500)
terrain = tt.Terrain.from_heightmap(h, [0.1, 0.1, 10])

terrain.spawn(physics_client_id)
pybullet.stepSimulation()

####### 
terrain.config.scale = [1, 1, 1]
terrain.config.save("./terrain_config.yaml")

######
import Config as conf

conf.save("./config.yaml")




pybullet build time: May  8 2025 04:00:00


startThreads creating 1 threads.
starting thread 0
Connected to PyBullet with client ID: 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (NVIDIA GeForce RTX 3060 Laptop GPU)
GL_VERSION=4.6 (Core Profile) Mesa 25.0.7-0ubuntu0.24.04.2
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 25.0.7-0ubuntu0.24.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (NVIDIA GeForce RTX 3060 Laptop GPU)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
Spawning Heightmap (Scale: [0.1, 0.1, 10], Origin: [0, 0, 0])...
ven = Microsoft Corpo

()

: 